https://realpython.com/k-means-clustering-python/

In [2]:
import tarfile
import urllib

import numpy as np
import matplotlib.pyplot as plt
import pandas as pd
import seaborn as sns

from sklearn.cluster import KMeans
from sklearn.decomposition import PCA
from sklearn.metrics import silhouette_score, adjusted_rand_score
from sklearn.pipeline import Pipeline
from sklearn.preprocessing import LabelEncoder, MinMaxScaler

## WYNIK BADANIA

Etykiety to ciągi znaków zawierające skróty typów nowotworów:

 - BRCA: Rak inwazyjny piersi
 - COAD: Gruczolakorak okrężnicy
 - KIRC: Rak jasnokomórkowy nerki
 - LUAD: Gruczolakorak płuc
 - PRAD: Gruczolakorak prostaty

In [20]:
df2 = pd.read_csv('/home/wmoszczy/Cluster_analysis/dane/labels.csv')
Af = df2.Class.to_list()
#Af
print(df2.Class.value_counts())
df2.head(3)

BRCA    300
KIRC    146
LUAD    141
PRAD    136
COAD     78
Name: Class, dtype: int64


,Unnamed: 0,Class
0,sample_0,PRAD
1,sample_1,LUAD
2,sample_2,PRAD


#### KODOWANIE WYNIKU aby go użyć...

In [26]:
df2['ClassK'] = pd.Categorical(df2.Class).codes
df2[['Class','ClassK']].head(8)

,Class,ClassK
0,PRAD,4
1,LUAD,3
2,PRAD,4
3,PRAD,4
4,BRCA,0
5,PRAD,4
6,KIRC,2
7,PRAD,4


#### SŁOWNIK

In [28]:
df2[['Class','ClassK']].value_counts()

Class  ClassK
BRCA   0         300
KIRC   2         146
LUAD   3         141
PRAD   4         136
COAD   1          78
dtype: int64

## ANALIZA GENOWA

In [17]:
df = pd.read_csv('/home/wmoszczy/Cluster_analysis/dane/data.csv')
df.head(4)

,Unnamed: 0,gene_0,gene_1,gene_2,gene_3,gene_4,gene_5,gene_6,gene_7,gene_8,...,gene_20521,gene_20522,gene_20523,gene_20524,gene_20525,gene_20526,gene_20527,gene_20528,gene_20529,gene_20530
0,sample_0,0.0,2.017209,3.265527,5.478487,10.431999,0.0,7.175175,0.591871,0.0,...,4.926711,8.210257,9.723516,7.220030,9.119813,12.003135,9.650743,8.921326,5.286759,0.0
1,sample_1,0.0,0.592732,1.588421,7.586157,9.623011,0.0,6.816049,0.000000,0.0,...,4.593372,7.323865,9.740931,6.256586,8.381612,12.674552,10.517059,9.397854,2.094168,0.0
2,sample_2,0.0,3.511759,4.327199,6.881787,9.870730,0.0,6.972130,0.452595,0.0,...,5.125213,8.127123,10.908640,5.401607,9.911597,9.045255,9.788359,10.090470,1.683023,0.0
3,sample_3,0.0,3.663618,4.507649,6.659068,10.196184,0.0,7.843375,0.434882,0.0,...,6.076566,8.792959,10.141520,8.942805,9.601208,11.392682,9.694814,9.684365,3.292001,0.0


## REDUKCJA WYMIAROWOŚCI
Przebadano tutaj ponad 20 tys genów, pacjentów jest 800 osób - każdy gen to wymiar, więc trzeba zredukować ilość genów (przekleństwo wymiarowości) 


In [30]:
df.shape

(801, 20532)

## Analiza składowych głównych (PCA) 
jest jedną z wielu technik redukcji wymiarowości. PCA przekształca dane wejściowe, rzutując je na mniejszą liczbę wymiarów zwanych komponentami . Komponenty wychwytują zmienność danych wejściowych poprzez liniową kombinację cech danych wejściowych.